In [2]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd




In [3]:
df_nf_unique = pd.read_csv('../df_nf_unique.csv')
df_nf_unique

/tmp/ipykernel_36921/418950838.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nf_unique = pd.read_csv('../df_nf_unique.csv')


,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
0,10,55,21,37005,2022-01-01 00:00:00.000,56228356013381,CRBS S/A - CDC Rio Branco,CDC RIO BRANCO,PESSOA_JURIDICA,NaN,...,"SKOL 600ML 6,...",Un,Un,2.0,2.0,5.69,5.69,7891149200405,7891149200405,22030000
1,14,55,21,37005,2022-01-01 00:00:00.000,56228356013381,CRBS S/A - CDC Rio Branco,CDC RIO BRANCO,PESSOA_JURIDICA,NaN,...,"ANTARCTICA PILSEN LATA 350ML SH C/12 NPAL 10,...",Un,Un,12.0,12.0,10.22,10.22,7891991002141,7891991000796,22030000
2,16,55,21,37005,2022-01-01 00:00:00.000,56228356013381,CRBS S/A - CDC Rio Branco,CDC RIO BRANCO,PESSOA_JURIDICA,NaN,...,"PEPSI COLA PET 2L CAIXA C/6 9,...",Un,Un,1.0,1.0,9.61,9.61,7892840800246,7892840800000,22021000
3,53,55,1,2148830,2022-01-01 00:00:00.000,3007331001032,EBAZAR.COM.BR. LTDA,NaN,PESSOA_FISICA,NaN,...,Potes Hermeticos Porta-Mantimentos Tampa Bambu...,UN,UN,1.0,1.0,139.90,139.90,7909569365786,7909569365786,70134290
4,88,55,1,25085,2022-01-01 00:00:00.000,21393527000101,RECOL FARMA LTDA.,RECOL FARMA - 24HORAS,PESSOA_JURIDICA,NaN,...,GRIPINEW C/20 CPR,UN1,UN1,1.0,1.0,8.99,8.99,7896862920029,7896862920029,30045090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248616,75679419,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,ROSQUINHA CHOCOLATE ZERO LOW 150G,UN1,UN1,1.0,1.0,8.07,8.07,7896292070080,7896292070080,19053100
2248617,75679436,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,BISC HOT CRACK QUEIJO 150G,UN1,UN1,8.0,8.0,6.26,6.26,7896011105499,7896011105499,19053100
2248618,75679441,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,MACARRAO NINHO LARGO PARATI 500G,UN1,UN1,1.0,1.0,5.25,5.25,7896011107028,7896011107028,19021100
2248619,75679480,55,10,12742,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,GRAFITE ESTAMPADAS,UN1,UN1,1.0,1.0,1.00,1.00,6944504411457,6944504411457,96084000


In [4]:
df_nf_unique.isnull().sum()

ID                                     0
MODELO                                 0
SERIE                                  0
NUMERO_NOTA_FISCAL                     0
DATA_EMISSAO                           0
CNPJ_CPF_EMITENTE                      0
RAZAO_SOCIAL_EMITENTE                  0
NOME_FANTASIA_EMITENTE            294074
TIPO_DESTINATARIO                      0
CNPJ_CPF_DESTINATARIO            2234472
CODIGO_MUNICIPIO_DESTINO               0
NUMERO_PRODUTO                         0
DESCRICAO_PRODUTO                      0
UNIDADE_PRODUTO                       26
UNIDADE_PRODUTO_TRIBUTAVEL             7
QUANTIDADE_PRODUTO                     0
QUANTIDADE_PRODUTO_TRIBUTAVEL          0
VALOR_PRODUTO                          0
VALOR_PRODUTO_TRIBUTAVEL               0
CODIGO_BARRA                           0
CODIGO_BARRA_TRIBUTAVEL                6
NCM                                    0
dtype: int64

# ElasticSearch


In [5]:
es = Elasticsearch(
    "http://localhost:9200",
    request_timeout=999999999,
)

es.info()

ObjectApiResponse({'name': 'melquicarvalho', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'VxyIZW2KQg-9GWvUtR9Qsw', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'deb', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

['index_t_nf', 'index_s']

In [8]:
codigos_repetidos = df_nf_unique[df_nf_unique['CODIGO_BARRA_TRIBUTAVEL'] == 'SEM GTIN']
codigos_repetidos

,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
62648,3100211,55,1,5289,2022-01-29 00:00:00.000,10373001000135,SUPERMERCADO TCHE LTDA ME,SUPERMERCADO TCHE,PESSOA_JURIDICA,NaN,...,COLHER DESC REFEIC SERT PLAST BCA 50UN,UN,UN,20.00,20.00,3.89,3.89,7898032372119,SEM GTIN,39241000
223078,21004643,55,3,1860,2022-06-21 00:00:00.000,529581000153,AUTO POSTO AMAPA EIRELI,AUTO POSTO AMAPA EIRELI,PESSOA_FISICA,NaN,...,DIESEL S-10 ADIT,LT,LT,262.86,262.86,5.39,5.39,7899050521244,SEM GTIN,27101921
224404,21113678,55,1,2106,2022-06-22 00:00:00.000,36158810000118,AUTO POSTO VIA VERDE LTDA,AUTO POSTO VIA VERDE,PESSOA_JURIDICA,NaN,...,DIESEL S-500 ADITIVADO,LTS,LTS,43.23,43.23,6.03,6.03,3000001900207,SEM GTIN,27101921
225311,21164935,55,0,116166,2022-06-22 00:00:00.000,13567147000100,S E G PETROLEO LTDA,AUTO POSTO CRUZEIRO,PESSOA_JURIDICA,NaN,...,DIESEL BS 500 ADITIVADO,L,L,60.70,60.70,8.95,8.95,3000001900207,SEM GTIN,27101259
231229,21523972,55,1,4132,2022-06-24 00:00:00.000,13567147000290,S & G PETROLEO LTDA,S & G PETROLEO LTDA,PESSOA_JURIDICA,NaN,...,DIESEL S500 ADITIVADO,L,L,200.02,200.02,8.95,8.95,3000001900207,SEM GTIN,27101921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165822,73359493,55,1,38830,2023-07-15 00:00:00.000,17337055000194,POSTO UNIAO DE LIMEIRA LTDA,POSTO UNIAO DE LIMEIRA,PESSOA_JURIDICA,NaN,...,5-OLEO DIESEL B S500 COMUM,L,L,201.60,201.60,4.59,4.59,3000001900207,SEM GTIN,27101921
2206872,74539906,55,1,117436,2023-07-24 00:00:00.000,548431000197,RADIN II LTDA,GAUCHAO 2,PESSOA_JURIDICA,NaN,...,Diesel B S10,l,L,893.52,893.52,5.25,5.25,7899050521244,SEM GTIN,27101921
2218539,74854269,55,1,7632,2023-07-26 00:00:00.000,376437000558,AUTO POSTO ABEL GALINHA LTDA 5,NaN,PESSOA_JURIDICA,NaN,...,OLEO DIESEL B S10 COMUM,LITRO,UNID,70.00,70.00,5.33,5.33,7899050521244,SEM GTIN,27101921
2239004,75426081,55,5,54034,2023-07-31 00:00:00.000,4313357000180,POSTO PAINEIRAO II LTDA,PAINEIRAO II,PESSOA_JURIDICA,NaN,...,DIESEL S10 ADITIVADO IPIMAX,L,L,70.00,70.00,5.09,5.09,7899050521244,SEM GTIN,27101921


In [9]:
def doc_generator(index_name, df):
  for idx, row in df.iterrows():
    yield {
        "_index": index_name,
        "_id": idx,
        "_source": {
            "modelo": row['MODELO'],
            "serie": row['SERIE'],
            "numero_nota_fiscal": row['NUMERO_NOTA_FISCAL'],
            "data_emissao": row['DATA_EMISSAO'],
            "cnpj_cpf_emitente": row['CNPJ_CPF_EMITENTE'],
            "razao_social_emitente": row['RAZAO_SOCIAL_EMITENTE'],
            "tipo_destinatario": row['TIPO_DESTINATARIO'],
            "valor_produto": row['VALOR_PRODUTO'],
            "codigo_barra": row['CODIGO_BARRA'],
            "ncm": row['NCM'],
            "descricao_produto": row['DESCRICAO_PRODUTO']
        }
    }

In [10]:
INDEX_NAME = "index_t_nf"
documentos_nf_index_generator = doc_generator(INDEX_NAME, df_nf_unique)

In [11]:
es.options(ignore_status=[400,404]).indices.delete(index=INDEX_NAME)

body = {
    "settings": {
        'number_of_shards': 3,
        'number_of_replicas': 1,
        "analysis": {
            "analyzer": {
                "standard_asciifolding": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [ "lowercase", "asciifolding" ]
                }
            }
        }
    },
    
    'mappings': {
        'properties': {
            'modelo': {'type': 'integer' },
            'serie': {'type': 'integer' },
            'numero_nota_fiscal': {'type': 'long' },
            'data_emissao': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss.SSS' },
            'cnpj_cpf_emitente': {'type': 'text' },
            'razao_social_emitente': {'type': 'text' },
            'tipo_destinatario': {'type': 'text' },
            'valor_produto': {'type': 'float' },
            'codigo_barra': {'type': 'long' },
            'ncm': {'type': 'long' },
            'descricao_produto': {'type': 'text', 'analyzer': 'brazilian', "fields": {
                                                                                        "acentuado": { 
                                                                                        "type":       "text",
                                                                                        "analyzer":   "standard_asciifolding"
                                                                                        }
                                                                                    }
                                }
        }
    }

}

es.indices.create(index=INDEX_NAME, body=body)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'index_t_nf'})

In [16]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

['index_t_nf', 'index_s']

In [13]:
try:
    response = helpers.bulk(es, documentos_nf_index_generator)

    if response[1]:
        errors = response[1]
        for error in errors:
            print("Falha de indexação:", error)
except helpers.BulkIndexError as e:
    for err in e.errors:
        print("Falha de indexação:", err)



In [14]:
doc_id = 1
resp = es.get(index=INDEX_NAME, id=doc_id)
print(resp)


{'_index': 'index_t_nf', '_id': '1', '_version': 1, '_seq_no': 1, '_primary_term': 1, 'found': True, '_source': {'modelo': 55, 'serie': 21, 'numero_nota_fiscal': 37005, 'data_emissao': '2022-01-01 00:00:00.000', 'cnpj_cpf_emitente': 56228356013381, 'razao_social_emitente': 'CRBS S/A - CDC Rio Branco', 'tipo_destinatario': 'PESSOA_JURIDICA', 'valor_produto': 10.22, 'codigo_barra': 7891991002141, 'ncm': 22030000, 'descricao_produto': 'ANTARCTICA PILSEN LATA 350ML SH C/12 NPAL  10,8333'}}


In [15]:
query = {
    "query": {
        "match": {
            "descricao_produto": "açucar 1kg cristal"
        }
    },
    "size": 16,
}



resp = es.search(index=INDEX_NAME, body=query)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['codigo_barra'])
    print(hit["_source"]['descricao_produto'])
    print('----------------------------')
    


Got 10000 Hits:
7896433800378
ACUCAR CRISTAL 1KG
----------------------------
7896433800378
ACUCAR CRISTAL  1KG
----------------------------
7896433800378
ACUCAR CRISTAL ITAMARATI 1KG
----------------------------
7891959014612
Acucar Cristal Uniao 1kg
----------------------------
7896433800378
ACUCAR ITAMARATI CRISTAL 1KG
----------------------------
7896433800378
ACUCAR ITAMARATI 1KG CRISTAL
----------------------------
7896433800378
ACUCAR CRISTAL 1KG ITAMARATI
----------------------------
7891959014612
ACUCAR UNIAO CRISTAL 1KG
----------------------------
7896433800378
AÇÚCAR CRISTAL ITAMARATI 1KG*
----------------------------
7896433800378
ACUCAR CRISTAL ITAMARATY 1KG
----------------------------
7891959014612
ACUCAR CRISTAL UNIAO 1KG
----------------------------
7897062500066
ACUCAR CRISTAL BARRALCOOL 1KG
----------------------------
7897062500066
ACUCAR BARRALCOOL CRISTAL 1KG
----------------------------
7896433800378
ACUCAR CRISTAL ITAMARATI- 1KG
----------------------------
789

In [17]:
query = {
    "query": {
        "match": {
            "descricao_produto": "fone wireless"
        }
    },
    "size": 16,
}


resp = es.search(index=INDEX_NAME, body=query)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['codigo_barra'])
    print(hit["_source"]['descricao_produto'])
    print('----------------------------')
    


Got 2446 Hits:
4893948974498
WIRELESS HEADSET FONE PR
----------------------------
7793617303208
FONE DE OUVIDO WIRELESS COM LED - PRATA
----------------------------
7793617312415
FONE DE OUVIDO WIRELESS COM LED - VERMELHO
----------------------------
846885010303
Fone True Wireless Earbuds - Champion
----------------------------
7908412811203
FONE DE OUVIDO WIRELESS HS-391 HREBOS
----------------------------
4893948974481
FONE DE OUV WIRELESS EARPHONE AZUL TMV
----------------------------
6925281997020
FONE TRUE WIRELESS JBL LIVEPRO2 PRATA NA
----------------------------
7891356092541
Fone De Ouvido Philco PFO03BTR Wireless Highbeat 22H
----------------------------
6925281953521
FONE TRUE WIRELESS JBL WAVE BUDS PRETO NA
----------------------------
7891143098404
Fone De Ouvido Wireless Bluetooth Inpods 12 Branco
----------------------------
7896276486098
Fone De Ouvido E6S True Wireless Headset Bluetooth 5.0
----------------------------
6972706510050
FONE DE OUVIDO SEM FIO WIRELESS ST

In [18]:
query_fuzziness = {
    "query": {
        "match": {
            "descricao_produto.acentuado": {
                "query": "feijo",
                "fuzziness": 1
            }
        }
    },
    "size": 16,
}

resp = es.search(index=INDEX_NAME, body=query_fuzziness)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['codigo_barra'])
    print(hit["_source"]['descricao_produto'])
    print('----------------------------')
    


Got 2370 Hits:
7898014400168
CAFE FEIJO 250G
----------------------------
7896183910525
FEIJAO
----------------------------
7896725341534
POTE MANTIMENTO FEIJAO POTE MANTIMENTO FEIJAO
----------------------------
7898014400168
CAFE FEIJO TORRADO E MOIDO
----------------------------
7898189491282
FEIJAO ZAIRE
----------------------------
7898935428043
FEIJAO CARIOQUINHA
----------------------------
7897326100339
FEIJAO BERNADO
----------------------------
7896327514343
SOPAO FEIJAO
----------------------------
7898189491367
FEIJÃO 1KG
----------------------------
7898189491428
FEIJAO  1KG
----------------------------
7896099100065
FEIJAO CARIOCA
----------------------------
7898189491428
FEIJAO 1KG
----------------------------
7896046600426
FEIJÃO MANTEIGUINHA
----------------------------
3000002608195
FEIJAO 01
----------------------------
7897326300036
Feijao manteiga
----------------------------
7896635195067
TEMP.MIKA FEIJAO
----------------------------


In [ ]:
query_phrase_analyzer = {
    "query": {
        "match_phrase": {
            "descricao_produto": "açucar cristalizado"
        }
    },
    "size": 8
}
resp = es.search(index=INDEX_NAME, body=query_phrase_analyzer)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['descricao_produto'])

Got 8 Hits:
ACUCAR CRISTALIZADO
AÇUCAR CRISTALIZADO 1KG
ACUCAR CRISTALIZADO PCT
ACUCAR CRISTALIZADO 1KG
AÇUCAR CRISTALIZADO  1KG
AÇUCAR CRISTALIZADO  ITAMARATI 1KG
ACUCAR CRISTALIZADO PCT DE 1KG
ACUCAR CRISTALIZADO ITAMARATI CRISTAL BRANCO 1KG FD-30UN CAMELI


In [ ]:
query_filter = {
    "query": {
        "bool": {
        "must": [
            {
            "match": {
                "descricao_produto": "doce de leite"
            }
            }
        ],
        "filter": [
            {
                "range": {
                    "data_emissao": {
                        "gte": "2023-01-01 00:00:00.000", 
                        "lte": "2023-01-31 00:00:00.000"                  
                    }

                },
                
            }
        ]
    }
  },
  'size': 20,
  
}

resp = es.search(index=INDEX_NAME, body=query_filter)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"])

Got 1065 Hits:
{'modelo': 55, 'serie': 1, 'numero_nota_fiscal': 2874, 'data_emissao': '2023-01-10 00:00:00.000', 'cnpj_cpf_emitente': 4607706000177, 'razao_social_emitente': 'CIOFFI & OLIVEIRA LTDA', 'tipo_destinatario': 'PESSOA_JURIDICA', 'valor_produto': 18.99, 'codigo_barra': 7898276513309, 'ncm': 19019020, 'descricao_produto': 'DOCE DE LEITE NERO 350G'}
{'modelo': 55, 'serie': 1, 'numero_nota_fiscal': 342, 'data_emissao': '2023-01-14 00:00:00.000', 'cnpj_cpf_emitente': 34125053000150, 'razao_social_emitente': 'MERCADO DO POVO LTDA', 'tipo_destinatario': 'PESSOA_JURIDICA', 'valor_produto': 18.49, 'codigo_barra': 7898276513309, 'ncm': 19019020, 'descricao_produto': 'DOCE DE LEITE CARTELA NERO'}
{'modelo': 55, 'serie': 1, 'numero_nota_fiscal': 39398, 'data_emissao': '2023-01-17 00:00:00.000', 'cnpj_cpf_emitente': 24396886002308, 'razao_social_emitente': 'COMERCIAL CENTERMIX LTDA', 'tipo_destinatario': 'PESSOA_JURIDICA', 'valor_produto': 27.45, 'codigo_barra': 7898952949309, 'ncm': 210

In [ ]:
query_fields = {
  "query": {
    "multi_match": {
      "query": "moto",
      "fields": ["descricao_produto", "razao_social_emitente"],
      "type": "cross_fields",
    }
  }
}

resp = es.search(index=INDEX_NAME, body=query_fields)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['descricao_produto'])
    print(hit["_source"]['razao_social_emitente'])
    print('---------------------')



Got 8859 Hits:
MINI MOTOS - 4 MOTOS 7896464736875
MANUFATURA DE BRINQUEDOS PICA-PAU LTDA
---------------------
MOT MOTO E40 GRAFITE N
ELLITE CELULAR LTDA
---------------------
MOTO-V LAVA MOTOS 1,5L
VOXER COMERCIO DE CERAS E POLIDORES LTDA
---------------------
MOTO-V LAVA MOTOS 500ML
VOXER COMERCIO DE CERAS E POLIDORES LTDA
---------------------
Moto-v lava motos 500ML
GLOSSY CAR COMERCIO DE ACESSORIOS LTDA
---------------------
MOTO-V LAVA MOTOS 1,5L VONIXX
VONIXX ACRE RIO BRANCO LTDA
---------------------
MOTO-V LAVA MOTOS 500ML VONIXX
VONIXX ACRE RIO BRANCO LTDA
---------------------
MC1000 MOTO MEDIDOR DE COMPRESSAO MOTO PLANATC.
GURGELMIX MAQUINAS E FERRAMENTAS S.A.
---------------------
MOTO DUO
CALESITA INDUSTRIA DE     BRINQUEDOS LTDA
---------------------
MOTO UNO
CALESITA INDUSTRIA DE     BRINQUEDOS LTDA
---------------------


In [19]:
query_pesos = {
  "query": {
    "multi_match": {
      "query": "moto",
      "fields": ["descricao_produto", "razao_social_emitente^2"],
      "type": "cross_fields",
    }
  }
}

resp = es.search(index=INDEX_NAME, body=query_pesos)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]["descricao_produto"])
    print(hit["_source"]["razao_social_emitente"])
    print("__________________")


Got 8859 Hits:
RETROVISOR MINI TITAN 2000 PADRAO HONDA DIR/ESQ
BOOMERANG MOTO PECAS
__________________
MANOPLA TITAN 150/2000 MODELO ORIGINAL
BOOMERANG MOTO PECAS
__________________
RETROVISOR YBR MODELO ORIGINAL DIR/ESQ
BOOMERANG MOTO PECAS
__________________
SUPORTE DO FAROL FAN 125 09/13 PRETO
BOOMERANG MOTO PECAS
__________________
BAGAGEIRO FAZER 150 MODELO 14/21 E FACTOR 150 16/21 PRETO
BOOMERANG MOTO PECAS
__________________
RETROVISOR YBR COM AJUSTE POR ESFERA PADRAO YAMAHA DIR/ESQ
BOOMERANG MOTO PECAS
__________________
PEDALEIRA TRASEIRA CG TITAN 125 MODELO 00/04
BOOMERANG MOTO PECAS
__________________
CAIXA DE DIRECAO C100 DREAM/BIZ/BIZ 125/WEB10
TOINHO MOTO PECAS
__________________
RETENTOR BENGALA CORTECO YAMAHA YBR 125
TOINHO MOTO PECAS
__________________
GUIDON TITAN 150 05/08 MODELO ORIGINAL POLIDO CROMADO
BOOMERANG MOTO PECAS
__________________


In [ ]:
query_pesos_diff = {
  "query": {
    "multi_match": {
      "query": "moto",
      "fields": ["descricao_produto^2", "razao_social_emitente"],
      "type": "cross_fields",
    }
  }
}

resp = es.search(index=INDEX_NAME, body=query_pesos_diff)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]["descricao_produto"])
    print(hit["_source"]["razao_social_emitente"])
    print("__________________")


In [ ]:
query_must = {
  "query": {
    "bool": {
            "must": [
                {"match": {"descricao_produto": "queijo"}},
            ],
            "must_not": [
                {"match": {"descricao_produto": "ralado"}}
            ],
    }
  },"size": 20
}
resp = es.search(index=INDEX_NAME, body=query_must)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['descricao_produto'])


Got 4178 Hits:
KIT QUEIJO ENTRE TACAS E QUEIJOS
QUEIJO
KRO QUEIJO SABOR QUEIJO 100G
QUEIJO TIPO KIT QUATRO QUEIJOS
PAO QUEIJO TIROLEZ 300G 3 QUEIJOS
QUEIJO MUSSARELA CASA QUEIJO 400G FAT.
SALG.BEBELA QUEIJO
SALG.TIME QUEIJO
SALG.LUKITAO QUEIJO
BOLO DE QUEIJO
QUEIJO COALHO
PAO DE QUEIJO
SALG.KRO QUEIJO
SALG.MIKITOS QUEIJO
SALG.AMAVITA QUEIJO
O queijo e os vermes
FATIADOR DE QUEIJO
KRO QUEIJO
QUEIJO MUSSARELA
AMENDUPA QUEIJO


In [ ]:
queryShould = {
  "query": {
    "bool": {
      "should": [
        { "match": { "descricao_produto": "farofa" }},
        { "match": { "descricao_produto": "bacon" }}
      ],
    }
  },"size": 26
}

resp = es.search(index=INDEX_NAME, body=queryShould)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['descricao_produto'])

Got 1025 Hits:
FAROFA DE BACON 500g
FAROFA MAND ACRE FAROFA 500G BACON
FAROFA APTI BACON 250GR
FAROFA GOURMET 500G BACON
FAROFA MIKA BACON 250G
FAROFA PRONTA BACON SINHA 10X250G
FAROFA GOURMET BACON KAMAR 280G
FAROFA PRONTA POLETTO 300G BACON
FAROFA PRONTA SINHA 250G BACON
Farofa Pronta Sinha 250g Bacon
FAROFA CASEIRA BACON 400GR 3730
FAROFA CASEIRA BACON 300 G
FAROFA CASEIRA BACON POLETTO 300 GR
FAROFA DU ACRE GOURMET 500G BACON
FAROFA MAND ACRE FAROFA 500G BACON UN QTD. 1.00 UN
POTE PARA FAROFA       POTE PARA FAROFA
FAROFA MAND ACRE FAROFA 500G ALHO
FAROFA DE ALHO 500G
FAROFA DE MANDIOCA CALABRESA
FAROFA DOCE VIAPANE
POTE PARA FAROFA
FAROFA CROCANTE DE AMENDOIM
FAROFA DE CALABRESA 500g
FAROFA ZAELI ORIGINAL
POTE EM PLASTICO FAROFA
FAROFA DE BANANA 500g
